Importamos librerías y funciones necesarias

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from ipynb.fs.full.Funciones_basicas import *
from ipynb.fs.full.Algoritmos import *
from ipynb.fs.full.Visualizaciones import *

In [3]:
from gurobipy import *
import random

Instanciamos el grafo

In [8]:
path_geom = '../data/graph_geom_corrected_cycles.csv'
geometry = gpd.read_file(path_geom, GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")
     
path = '../data/corrected_dijkstra.csv'
csv_grafo = pd.read_csv(path, sep=',')

path_pesos = '../data/pesos_tapas.csv'
pesos = pd.read_csv(path_pesos)

out = {1003950, 15004, 15131, 1003744, 15190, 1003746, 13730, 1003167, 13731, 13732, 13733, 14062, 1003206, 16503, 13735, 16094, 16095}

S = set()
for index, row in csv_grafo.iterrows():
    origin = row['self']
    dest = row['other']
    if origin not in out and dest not in out:
        S.add(origin)
        S.add(dest)

S = list(S)
id_ = {}; _id = {}; l = 0
for u in S:
    id_[u] = l; _id[l] = u
    l += 1

T = nx.DiGraph()
for index, row in csv_grafo.iterrows():
    origin = row['self']
    dest = row['other']
    if origin not in out and dest not in out:
        T.add_edge(id_[origin], id_[dest])
    
N = l

W = [0] * N

for u in range(N):
    if pesos[pesos['ID_tapa'] == _id[u]].shape[0] >= 1:  ## tomamos primera columna con el id, si no hay peso = 0
        W[u] = pesos[pesos['ID_tapa'] == _id[u]].iloc[0]['per_predio']

Definimos conjuntos e instanciamos el modelo

In [10]:
k = 12
r = 744
K_ = range(1,k)

In [9]:
V = list(T.nodes())
I = {(u, v): u in get_ideal(T, [0] * N, v) for u in V for v in V}
K_ = range(1,k)

In [12]:
#model = gurobipy.Model("Muestreo")

Definimos las variables

In [ ]:
x = model.addVars(V, K_, vtype = GRB.BINARY, name ="Nodos raíces")
alpha_hat = model.addVars(K_, K_, vtype = GRB.BINARY, name ="Ideal hijo")
alpha = model.addVars(K_, K_, vtype = GRB.BINARY, name ="Ideal hijo directo")

Detallamos las restricciones

In [ ]:
model.addConstrs((quicksum(x[v,k]  for v in V) == 1 for k in K_), name = "R1")
model.addConstrs((quicksum(x[v,k]  for k in K_) <= 1 for v in V), name = "R2")
model.addConstrs((alpha_hat[r,k] >= (x[v,r] + (x[u,k] + I[v,u] - 2 for v in V for u in V for r in K_ for k in K_ ), name = "R3")

In [ ]:
model.addConstrs((alpha[r,t] <= 2 - (alpha_hat[r,k] + alpha_hat[k,t]) for r in K_ for k in K_ for t in K_), name = "R4")
model.addConstrs((quicksum(quicksum(alpha[k,t]  for k in K_) for t in K_) == k - 1), name = "R5")
model.addConstrs((x[r,1] == 1), name = "R6")

Definimos la función objetivo

In [ ]:
obj = quicksum(alpha[i,t]*x[i,t] + quicksum(c[i,j]*y[i,j,t] for j in M_) for i in N_ for t in T_)
model.setObjective(obj, GRB.MINIMIZE)

In [ ]:
model.optimize()